In [ ]:
!pip install neo4j requests

In [ ]:
import json
import time
from google.colab import userdata
from neo4j import GraphDatabase
import requests
from datetime import datetime
from typing import List, Dict, Any

# Neo4j credentials.
NEO4J_URI = userdata.get('NEO4J_URI')
NEO4J_USER = userdata.get('NEO4J_USER')
NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')

# Polymarket Gamma API.
GAMMA_API_BASE = 'https://gamma-api.polymarket.com'

print('Configuration loaded successfully')

In [ ]:
# Initialize Neo4j driver.
neo4j_driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
print('✓ Neo4j driver initialized')

# Test Gamma API connection.
try:
    response = requests.get(f'{GAMMA_API_BASE}/markets', timeout=10)
    if response.status_code == 200:
        print('✓ Connected to Polymarket Gamma API')
    else:
        print(f'✗ Gamma API returned status code: {response.status_code}')
except Exception as e:
    print(f'✗ Error connecting to Gamma API: {e}')

# Test Blockscout API connection.
try:
    response = requests.get('https://polygon.blockscout.com/api/v2/stats', timeout=10)
    if response.status_code == 200:
        print('✓ Connected to Blockscout API (Polygon)')
    else:
        print(f'✗ Blockscout API returned status code: {response.status_code}')
except Exception as e:
    print(f'✗ Error connecting to Blockscout API: {e}')

print('\n✅ All clients initialized successfully')

In [ ]:
def fetch_latest_events(limit: int = 50) -> List[Dict[str, Any]]:
    """Fetch the latest active events from Polymarket Gamma API."""
    try:
        url = f'{GAMMA_API_BASE}/events'
        params = {
            'limit': limit,
            'offset': 0,
            'closed': 'true',
            'order': 'id',
            'ascending': 'false'
        }

        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()

        return response.json()

    except Exception as e:
        print(f'Error fetching events: {e}')
        return []

# Fetch events (including closed ones since token transfers are historical).
latest_events = fetch_latest_events(10)
total_markets = sum(len(event.get('markets', [])) for event in latest_events)

print(f'Fetched {len(latest_events)} events and {total_markets} markets.')

In [ ]:
import json

def extract_category_from_tags(event: Dict[str, Any]) -> str:
    """Extract category from event tags."""
    if event.get('category'):
        return event['category']

    tags = event.get('tags', [])
    priority_categories = ['Sports', 'Politics', 'Finance', 'Crypto', 'Science', 'Entertainment']

    for tag in tags:
        label = tag.get('label', '')
        if label in priority_categories:
            return label

    for tag in tags:
        label = tag.get('label', '')
        if label and label not in ['All', 'Hide From New', 'Daily', 'Recurring']:
            return label

    return 'Unknown'

def extract_outcomes_from_market(market: Dict[str, Any]) -> List[Dict[str, Any]]:
    """Extract outcomes with token IDs and prices from market."""
    outcomes_str = market.get('outcomes', '[]')
    outcome_names = json.loads(outcomes_str) if isinstance(outcomes_str, str) else (outcomes_str or [])

    prices_str = market.get('outcomePrices', '[]')
    prices = json.loads(prices_str) if isinstance(prices_str, str) else (prices_str or [])

    token_ids_str = market.get('clobTokenIds', '[]')
    token_ids = json.loads(token_ids_str) if isinstance(token_ids_str, str) else (token_ids_str or [])

    outcomes = []
    for i, outcome_name in enumerate(outcome_names):
        outcomes.append({
            'token_id': token_ids[i] if i < len(token_ids) else f"unknown_{market['id']}_{i}",
            'name': outcome_name,  # Changed from 'outcome_name' to 'name'
            'price': prices[i] if i < len(prices) else '0.5',  # Changed from 'current_price' to 'price'
            'condition_id': market.get('conditionId'),  # Added for linking trades to markets
        })

    return outcomes

# Process events.
print('Processing events...')
for event in latest_events:
    event['category'] = extract_category_from_tags(event)

categories = set(event['category'] for event in latest_events)
print(f'  Categories: {", ".join(categories)}')

# Extract outcomes.
print('Extracting outcomes and condition IDs...')
all_outcomes = []
all_condition_ids = set()

for event in latest_events:
    for market in event.get('markets', []):
        outcomes = extract_outcomes_from_market(market)
        all_outcomes.extend(outcomes)

        # Collect condition IDs for trade filtering.
        condition_id = market.get('conditionId')
        if condition_id:
            all_condition_ids.add(condition_id)

print(f'  Outcomes: {len(all_outcomes)} (from {total_markets} markets)')
print(f'  Unique Condition IDs: {len(all_condition_ids)}')

# Collect all token IDs for later (not used for blockchain matching anymore).
outcome_token_ids = set(outcome['token_id'] for outcome in all_outcomes if not outcome['token_id'].startswith('unknown_'))
print(f'  Valid Token IDs: {len(outcome_token_ids)}')

# USDC address.
usdc_address = '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174'

print('\n✅ Polymarket data processed')

In [ ]:
def fetch_trades_from_data_api(condition_ids: List[str] = None, max_trades: int = None, batch_desc: str = '') -> List[Dict[str, Any]]:
    """Fetch ALL trades from Polymarket Data API with pagination."""
    import time as time_module
    from tqdm.notebook import tqdm

    url = 'https://data-api.polymarket.com/trades'
    all_trades = []
    offset = 0
    page_size = 500  # Polymarket API hard limit is 500 per request

    # Create a progress bar without a known total (will just count up)
    pbar = tqdm(total=None, desc=f'{batch_desc}', unit=' trades', position=1, leave=False, bar_format='{desc}: {n_fmt} trades fetched')

    try:
        while True:
            params = {
                'limit': page_size,
                'offset': offset,
                'takerOnly': 'true'
            }

            # Add market filter if condition IDs provided.
            if condition_ids:
                params['market'] = ','.join(condition_ids)

            response = requests.get(url, params=params, timeout=30)

            if response.status_code != 200:
                pbar.write(f'    ⚠ Error: HTTP {response.status_code}')
                break

            trades = response.json()

            if len(trades) == 0:
                break

            # Transform to our format and extract ALL data from Polymarket API.
            for trade in trades:
                all_trades.append({
                    'hash': trade.get('transactionHash', ''),
                    'from': trade.get('proxyWallet', ''),
                    'to': '',
                    'side': trade.get('side', ''),
                    'condition_id': trade.get('conditionId', ''),
                    'outcome': trade.get('outcome', ''),
                    'outcome_index': trade.get('outcomeIndex', 0),
                    'size': float(trade.get('size', 0)),
                    'price': float(trade.get('price', 0)),
                    'timestamp': trade.get('timestamp', 0),
                    'asset': trade.get('asset', ''),
                    'market_slug': trade.get('slug', ''),
                    'market_title': trade.get('title', ''),
                    'market_icon': trade.get('icon', ''),
                    'event_slug': trade.get('eventSlug', ''),
                    'user_name': trade.get('name', ''),
                    'user_pseudonym': trade.get('pseudonym', ''),
                    'user_bio': trade.get('bio', ''),
                    'user_profile_image': trade.get('profileImage', ''),
                    'user_profile_image_optimized': trade.get('profileImageOptimized', ''),
                })

            # Update progress bar with new trade count
            pbar.n = len(all_trades)
            pbar.refresh()

            # Check if we've reached the max trades limit.
            if max_trades and len(all_trades) >= max_trades:
                all_trades = all_trades[:max_trades]
                pbar.write(f'    ℹ Reached limit of {max_trades:,} trades for this batch')
                break

            # Check if we got fewer trades than the page size (end of data).
            if len(trades) < page_size:
                break

            # Move to next page.
            offset += len(trades)

            # Rate limiting to avoid hitting API limits.
            time_module.sleep(0.3)

        pbar.close()
        return all_trades

    except Exception as e:
        pbar.close()
        print(f'    ⚠ Error: {e}')
        return all_trades

from tqdm.notebook import tqdm

print('Fetching ALL trades from Polymarket Data API...')
print(f'Strategy: Fetch trades per market individually (max 10,000 trades per market)\n')

# Fetch trades per market individually to ensure fair distribution
condition_id_list = list(all_condition_ids)
all_token_transfers = []

print(f'Total markets: {len(condition_id_list)}')
print(f'Limit: 10,000 trades per market (to prevent single active markets from dominating)\n')

# Create progress bar for markets
for i, condition_id in enumerate(tqdm(condition_id_list, desc='Fetching Markets', unit='market', position=0)):
    market_desc = f'Market {i+1}/{len(condition_id_list)}'

    # Fetch trades for single market with 10,000 trade limit
    trades = fetch_trades_from_data_api(condition_ids=[condition_id], max_trades=10000, batch_desc=market_desc)
    all_token_transfers.extend(trades)

print(f'\n✅ Completed!')

print(f'\n📊 Total: Fetched {len(all_token_transfers)} trades\n')

if len(all_token_transfers) > 0:
    # Get unique condition IDs from trades.
    fetched_condition_ids = set(t['condition_id'] for t in all_token_transfers if t['condition_id'])

    # Get unique users.
    users = set(t['from'] for t in all_token_transfers if t['from'])

    # Calculate total volume.
    total_volume = sum(t['size'] for t in all_token_transfers)

    print(f'📊 Trade Statistics:')
    print(f'─' * 70)
    print(f'  Total Trades:         {len(all_token_transfers):,}')
    print(f'  Markets Requested:    {len(all_condition_ids)}')
    print(f'  Markets with Trades:  {len(fetched_condition_ids)}')
    print(f'  Unique Users:         {len(users):,}')
    print(f'  Total Volume:         ${total_volume:,.2f} USDC')
    if len(all_token_transfers) > 0:
        print(f'  Average Trade Size:   ${total_volume / len(all_token_transfers):,.2f} USDC')
    print(f'─' * 70)

    # Check for markets with no trades
    missing_condition_ids = all_condition_ids - fetched_condition_ids
    if len(missing_condition_ids) > 0:
        print(f'\n⚠️  Warning: {len(missing_condition_ids)} markets returned NO trades:')
        print(f'   This could mean:')
        print(f'   • Markets are too old (API might not have historical data)')
        print(f'   • Markets had no trading activity')
        print(f'   • API filtering issue with those condition IDs')
        print(f'\n   Missing condition IDs (first 5):')
        for cid in list(missing_condition_ids)[:5]:
            print(f'   • {cid}')
        if len(missing_condition_ids) > 5:
            print(f'   ... and {len(missing_condition_ids) - 5} more')

    print(f'\n✅ Successfully collected {len(all_token_transfers):,} trades for our markets!')
else:
    print('\n⚠️  WARNING: No trades fetched!')
    print('    Possible reasons:')
    print('    - Markets are too new/old')
    print('    - API filter syntax issue')
    print('    - Markets have no trading activity')


In [ ]:
import json
from datetime import datetime
from tqdm.notebook import tqdm

print('=' * 70)
print('Importing Data to Neo4j')
print('=' * 70)
print()

# Prepare user data from trades with Polymarket profile data.
print('[0/10] Preparing user data...')
user_profiles = {}
null_address = '0x0000000000000000000000000000000000000000'

for trade in all_token_transfers:
    trader_addr = trade.get('from')
    if not trader_addr or trader_addr == null_address:
        continue

    if trader_addr not in user_profiles:
        user_profiles[trader_addr] = {
            'address': trader_addr,
            'role': 'trader',
            'name': trade.get('user_name', ''),
            'pseudonym': trade.get('user_pseudonym', ''),
            'bio': trade.get('user_bio', ''),
            'profile_image': trade.get('user_profile_image', ''),
            'profile_image_optimized': trade.get('user_profile_image_optimized', ''),
        }

print(f'  ✓ Prepared {len(user_profiles)} users\n')

def create_schema(driver):
    """Create constraints and indexes."""
    print('[1/9] Creating schema...')

    statements = [
        # Constraints.
        'CREATE CONSTRAINT user_address IF NOT EXISTS FOR (u:User) REQUIRE u.address IS UNIQUE',
        'CREATE CONSTRAINT event_slug IF NOT EXISTS FOR (e:Event) REQUIRE e.slug IS UNIQUE',
        'CREATE CONSTRAINT market_condition_id IF NOT EXISTS FOR (m:Market) REQUIRE m.condition_id IS UNIQUE',
        'CREATE CONSTRAINT outcome_id IF NOT EXISTS FOR (o:Outcome) REQUIRE (o.condition_id, o.outcome_index) IS UNIQUE',
        'CREATE CONSTRAINT trade_hash IF NOT EXISTS FOR (t:Trade) REQUIRE t.transaction_hash IS UNIQUE',

        # Indexes.
        'CREATE INDEX event_category IF NOT EXISTS FOR (e:Event) ON (e.category)',
        'CREATE INDEX event_closed IF NOT EXISTS FOR (e:Event) ON (e.closed)',
        'CREATE INDEX market_slug IF NOT EXISTS FOR (m:Market) ON (m.slug)',
        'CREATE INDEX trade_timestamp IF NOT EXISTS FOR (t:Trade) ON (t.timestamp)',
        'CREATE INDEX trade_side IF NOT EXISTS FOR (t:Trade) ON (t.side)',
        'CREATE INDEX user_role IF NOT EXISTS FOR (u:User) ON (u.role)',
    ]

    with driver.session() as session:
        for stmt in statements:
            try:
                session.run(stmt)
            except Exception:
                pass  # Constraint/index may already exist

    print('  ✓ Schema created\n')

def clear_database(driver):
    """Clear all data."""
    print('[2/9] Clearing database...')
    with driver.session() as session:
        session.run('MATCH (n) DETACH DELETE n')
    print('  ✓ Database cleared\n')

def import_events(driver, events):
    """Import Event nodes in batch."""
    print('[3/9] Importing events...')

    # Prepare data.
    event_data = []
    for event in events:
        tags = event.get('tags', [])
        tag_labels = [tag.get('label', '') for tag in tags if tag.get('label')]

        event_data.append({
            'slug': event.get('slug'),
            'title': event.get('title', ''),
            'description': event.get('description', ''),
            'category': event.get('category', 'Unknown'),
            'start_date': event.get('startDate', '2020-01-01T00:00:00Z'),
            'end_date': event.get('endDate', '2030-01-01T00:00:00Z'),
            'closed': event.get('closed', False),
            'volume': event.get('volume', 0),
            'liquidity': event.get('liquidity', 0) if event.get('liquidity') else 0,
            'open_interest': event.get('openInterest', 0),
            'icon': event.get('icon', ''),
            'image': event.get('image', ''),
            'comment_count': event.get('commentCount', 0),
            'tags': tag_labels,
            'restricted': event.get('restricted', False),
            'featured': event.get('featured', False),
        })

    # Batch import using UNWIND.
    with driver.session() as session:
        session.run('''
            UNWIND $events as event
            MERGE (e:Event {slug: event.slug})
            SET e.title = event.title,
                e.description = event.description,
                e.category = event.category,
                e.start_date = datetime(event.start_date),
                e.end_date = datetime(event.end_date),
                e.closed = event.closed,
                e.volume = toFloat(event.volume),
                e.liquidity = toFloat(event.liquidity),
                e.open_interest = toFloat(event.open_interest),
                e.icon = event.icon,
                e.image = event.image,
                e.comment_count = toInteger(event.comment_count),
                e.tags = event.tags,
                e.restricted = event.restricted,
                e.featured = event.featured
        ''', {'events': event_data})

    print(f'  ✓ Imported {len(events)} events\n')

def import_markets(driver, events):
    """Import Market nodes in batches."""
    print('[4/9] Importing markets...')

    # Prepare market data.
    market_data = []

    for event in events:
        event_slug = event.get('slug')

        for market in event.get('markets', []):
            condition_id = market.get('conditionId')
            if not condition_id:
                continue

            # Determine resolution status.
            closed = market.get('closed', False)
            uma_resolution_status = market.get('umaResolutionStatus', '')
            resolved = uma_resolution_status == 'resolved'
            winning_outcome = None

            if resolved:
                outcome_prices_str = market.get('outcomePrices', '[]')
                try:
                    outcome_prices = json.loads(outcome_prices_str) if isinstance(outcome_prices_str, str) else outcome_prices_str
                    outcomes_str = market.get('outcomes', '[]')
                    outcome_names = json.loads(outcomes_str) if isinstance(outcomes_str, str) else outcomes_str

                    for i, price in enumerate(outcome_prices):
                        if float(price) >= 0.99:
                            winning_outcome = outcome_names[i] if i < len(outcome_names) else None
                            break
                except Exception:
                    pass

            # Convert closedTime format.
            closed_time = market.get('closedTime', '')
            if closed_time:
                try:
                    closed_time = closed_time.replace(' ', 'T').replace('+00', 'Z')
                except Exception:
                    closed_time = '2020-01-01T00:00:00Z'
            else:
                closed_time = '2020-01-01T00:00:00Z'

            market_data.append({
                'condition_id': condition_id,
                'question': market.get('question', ''),
                'slug': market.get('slug', ''),
                'description': market.get('description', ''),
                'question_id': market.get('questionID', ''),
                'start_date': market.get('startDate', '2020-01-01T00:00:00Z'),
                'end_date': market.get('endDate', '2030-01-01T00:00:00Z'),
                'closed': closed,
                'closed_time': closed_time,
                'resolved': resolved,
                'winning_outcome': winning_outcome,
                'resolved_by': market.get('resolvedBy', ''),
                'uma_resolution_status': uma_resolution_status,
                'volume': market.get('volumeNum', 0),
                'volume_clob': market.get('volumeClob', 0),
                'liquidity': market.get('liquidityNum', 0) if market.get('liquidityNum') else 0,
                'last_trade_price': market.get('lastTradePrice', 0),
                'best_ask': market.get('bestAsk', 0),
                'best_bid': market.get('bestBid', 0),
                'spread': market.get('spread', 0),
                'neg_risk': market.get('negRisk', False),
                'neg_risk_market_id': market.get('negRiskMarketID'),
                'group_item_title': market.get('groupItemTitle'),
                'group_item_threshold': market.get('groupItemThreshold'),
                'restricted': market.get('restricted', False),
                'active': market.get('active', True),
                'event_slug': event_slug,
            })

    # Batch import markets.
    with driver.session() as session:
        session.run('''
            UNWIND $markets as market
            MERGE (m:Market {condition_id: market.condition_id})
            SET m.question = market.question,
                m.slug = market.slug,
                m.description = market.description,
                m.question_id = market.question_id,
                m.start_date = datetime(market.start_date),
                m.end_date = datetime(market.end_date),
                m.closed = market.closed,
                m.closed_time = datetime(market.closed_time),
                m.resolved = market.resolved,
                m.winning_outcome = market.winning_outcome,
                m.resolved_by = market.resolved_by,
                m.uma_resolution_status = market.uma_resolution_status,
                m.volume = toFloat(market.volume),
                m.volume_clob = toFloat(market.volume_clob),
                m.liquidity = toFloat(market.liquidity),
                m.last_trade_price = toFloat(market.last_trade_price),
                m.best_ask = toFloat(market.best_ask),
                m.best_bid = toFloat(market.best_bid),
                m.spread = toFloat(market.spread),
                m.neg_risk = market.neg_risk,
                m.neg_risk_market_id = market.neg_risk_market_id,
                m.group_item_title = market.group_item_title,
                m.group_item_threshold = market.group_item_threshold,
                m.restricted = market.restricted,
                m.active = market.active
        ''', {'markets': market_data})

        # Create Market -> Event relationships.
        session.run('''
            UNWIND $markets as market
            MATCH (m:Market {condition_id: market.condition_id})
            MATCH (e:Event {slug: market.event_slug})
            MERGE (m)-[:PART_OF_EVENT]->(e)
        ''', {'markets': market_data})

    print(f'  ✓ Imported {len(market_data)} markets\n')

def import_outcomes(driver, events):
    """Import Outcome nodes in batches."""
    print('[5/9] Importing outcomes...')

    # Prepare outcome data.
    outcome_data = []

    for event in events:
        for market in event.get('markets', []):
            condition_id = market.get('conditionId')
            if not condition_id:
                continue

            # Parse outcomes.
            outcomes_str = market.get('outcomes', '[]')
            outcome_names = json.loads(outcomes_str) if isinstance(outcomes_str, str) else (outcomes_str or [])

            prices_str = market.get('outcomePrices', '[]')
            prices = json.loads(prices_str) if isinstance(prices_str, str) else (prices_str or [])

            token_ids_str = market.get('clobTokenIds', '[]')
            token_ids = json.loads(token_ids_str) if isinstance(token_ids_str, str) else (token_ids_str or [])

            for i, outcome_name in enumerate(outcome_names):
                outcome_data.append({
                    'condition_id': condition_id,
                    'outcome_index': i,
                    'outcome_name': outcome_name,
                    'current_price': prices[i] if i < len(prices) else 0.5,
                    'token_id': token_ids[i] if i < len(token_ids) else '',
                })

    # Batch import outcomes.
    with driver.session() as session:
        session.run('''
            UNWIND $outcomes as outcome
            MERGE (o:Outcome {condition_id: outcome.condition_id, outcome_index: outcome.outcome_index})
            SET o.outcome_name = outcome.outcome_name,
                o.current_price = toFloat(outcome.current_price),
                o.token_id = outcome.token_id
        ''', {'outcomes': outcome_data})

        # Create Market -> Outcome relationships.
        session.run('''
            UNWIND $outcomes as outcome
            MATCH (o:Outcome {condition_id: outcome.condition_id, outcome_index: outcome.outcome_index})
            MATCH (m:Market {condition_id: outcome.condition_id})
            MERGE (m)-[:HAS_OUTCOME]->(o)
        ''', {'outcomes': outcome_data})

    print(f'  ✓ Imported {len(outcome_data)} outcomes\n')

def import_users(driver, users):
    """Import User nodes with Polymarket profile data."""
    print('[6/9] Importing users...')

    # Prepare user data.
    user_data = []
    for user in users.values():
        user_data.append({
            'address': user['address'],
            'role': 'trader',
            'name': user.get('name', ''),
            'pseudonym': user.get('pseudonym', ''),
            'bio': user.get('bio', ''),
            'profile_image': user.get('profile_image', ''),
            'profile_image_optimized': user.get('profile_image_optimized', ''),
        })

    # Batch import users.
    with driver.session() as session:
        session.run('''
            UNWIND $users as user
            MERGE (u:User {address: user.address})
            SET u.role = user.role,
                u.name = user.name,
                u.pseudonym = user.pseudonym,
                u.bio = user.bio,
                u.profile_image = user.profile_image,
                u.profile_image_optimized = user.profile_image_optimized
        ''', {'users': user_data})

    print(f'  ✓ Imported {len(users)} users\n')

def import_trades(driver, trades):
    """Import Trade nodes in batches for speed."""
    print('[7/9] Importing trades...')

    batch_size = 500  # Process 500 trades at a time
    total_batches = (len(trades) + batch_size - 1) // batch_size

    imported_count = 0
    skipped_count = 0

    with driver.session() as session:
        for batch_idx in tqdm(range(0, len(trades), batch_size), desc='  Trade Batches', unit='batch', total=total_batches):
            batch = trades[batch_idx:batch_idx + batch_size]

            # Prepare batch data.
            trade_data = []
            for trade in batch:
                tx_hash = trade.get('hash')
                condition_id = trade.get('condition_id')
                trader_address = trade.get('from')

                if not tx_hash or not condition_id or not trader_address:
                    skipped_count += 1
                    continue

                # Parse timestamp.
                timestamp_value = trade.get('timestamp')
                if timestamp_value:
                    try:
                        timestamp_iso = datetime.fromtimestamp(int(timestamp_value)).isoformat()
                    except Exception:
                        timestamp_iso = datetime.now().isoformat()
                else:
                    timestamp_iso = datetime.now().isoformat()

                trade_data.append({
                    'transaction_hash': tx_hash,
                    'timestamp': timestamp_iso,
                    'side': trade.get('side', 'BUY'),
                    'size_usdc': trade.get('size', 0),
                    'price': trade.get('price', 0),
                    'outcome_name': trade.get('outcome', ''),
                    'outcome_index': trade.get('outcome_index', 0),
                    'market_title': trade.get('market_title', ''),
                    'market_slug': trade.get('market_slug', ''),
                    'event_slug': trade.get('event_slug', ''),
                    'trader_address': trader_address,
                    'condition_id': condition_id,
                })

            if not trade_data:
                continue

            # Batch import trades.
            session.run('''
                UNWIND $trades as trade
                MERGE (t:Trade {transaction_hash: trade.transaction_hash})
                SET t.timestamp = datetime(trade.timestamp),
                    t.side = trade.side,
                    t.size_usdc = toFloat(trade.size_usdc),
                    t.price = toFloat(trade.price),
                    t.outcome_name = trade.outcome_name,
                    t.outcome_index = toInteger(trade.outcome_index),
                    t.market_title = trade.market_title,
                    t.market_slug = trade.market_slug,
                    t.event_slug = trade.event_slug
            ''', {'trades': trade_data})

            # Batch create User->Trade relationships.
            session.run('''
                UNWIND $trades as trade
                MATCH (t:Trade {transaction_hash: trade.transaction_hash})
                MATCH (u:User {address: trade.trader_address})
                MERGE (u)-[:PLACED_TRADE]->(t)
            ''', {'trades': trade_data})

            # Batch create Trade->Market relationships.
            session.run('''
                UNWIND $trades as trade
                MATCH (t:Trade {transaction_hash: trade.transaction_hash})
                MATCH (m:Market {condition_id: trade.condition_id})
                MERGE (t)-[:ON_MARKET]->(m)
            ''', {'trades': trade_data})

            # Batch create Trade->Outcome relationships.
            session.run('''
                UNWIND $trades as trade
                MATCH (t:Trade {transaction_hash: trade.transaction_hash})
                MATCH (o:Outcome {condition_id: trade.condition_id, outcome_index: trade.outcome_index})
                MERGE (t)-[:FOR_OUTCOME]->(o)
            ''', {'trades': trade_data})

            imported_count += len(trade_data)

    print(f'  ✓ Imported {imported_count} trades ({skipped_count} skipped)\n')

def create_group_market_relationships(driver, events):
    """Link markets in the same group."""
    print('[8/9] Creating group market relationships...')

    count = 0

    with driver.session() as session:
        for event in events:
            for market in event.get('markets', []):
                neg_risk_market_id = market.get('negRiskMarketID')
                condition_id = market.get('conditionId')

                if neg_risk_market_id and condition_id:
                    session.run('''
                        MATCH (m1:Market {condition_id: $condition_id})
                        MATCH (m2:Market)
                        WHERE m2.neg_risk_market_id = $neg_risk_market_id
                          AND m1.condition_id <> m2.condition_id
                        MERGE (m1)-[:SAME_GROUP {group_id: $neg_risk_market_id}]->(m2)
                    ''', {
                        'condition_id': condition_id,
                        'neg_risk_market_id': neg_risk_market_id,
                    })
                    count += 1

    print(f'  ✓ Created relationships for {count} group markets\n')

def create_holdings(driver):
    """Calculate user holdings from BUY trades."""
    print('[9/9] Creating holdings...')

    with driver.session() as session:
        result = session.run('''
            MATCH (u:User)-[:PLACED_TRADE]->(t:Trade)-[:FOR_OUTCOME]->(o:Outcome)
            WHERE t.side = 'BUY'
            WITH u, o, sum(t.size_usdc) as total_invested, max(t.timestamp) as last_trade
            MERGE (u)-[h:HOLDS]->(o)
            SET h.invested_usdc = total_invested,
                h.last_updated = last_trade
            RETURN count(*) as holdings_created
        ''')

        count = result.single()['holdings_created']

    print(f'  ✓ Created {count} holdings\n')

# Main execution.
print('Starting Neo4j import...\n')

create_schema(neo4j_driver)
clear_database(neo4j_driver)
import_events(neo4j_driver, latest_events)
import_markets(neo4j_driver, latest_events)
import_outcomes(neo4j_driver, latest_events)
import_users(neo4j_driver, user_profiles)
import_trades(neo4j_driver, all_token_transfers)
create_group_market_relationships(neo4j_driver, latest_events)
create_holdings(neo4j_driver)

print('=' * 70)
print('✅ Data import complete!')
print('=' * 70)
print()
print('Summary:')
print(f'  • Events: {len(latest_events)}')
print(f'  • Markets: {total_markets}')
print(f'  • Outcomes: {len(all_outcomes)}')
print(f'  • Trades: {len(all_token_transfers)}')
print(f'  • Users: {len(user_profiles)}')
print('=' * 70)


In [ ]:
from tqdm.notebook import tqdm

print('=' * 70)
print('Verifying Neo4j Database')
print('=' * 70)
print()

def verify_database(driver):
    """Run comprehensive verification queries."""

    with driver.session() as session:
        # ====================================================================
        # 1. NODE COUNTS
        # ====================================================================
        print('[1/7] Counting Nodes...')
        print('-' * 70)

        node_types = ['Event', 'Market', 'Outcome', 'User', 'Contract', 'Trade']
        node_counts = {}

        for node_type in tqdm(node_types, desc='Node Types'):
            result = session.run(f'MATCH (n:{node_type}) RETURN count(n) as count')
            node_counts[node_type] = result.single()['count']
            print(f'  • {node_type}: {node_counts[node_type]:,}')

        # Specific counts.
        result = session.run('MATCH (u:User) WHERE u.role = "trader" RETURN count(u) as count')
        trader_count = result.single()['count']
        print(f'  • Traders (subset of Users): {trader_count:,}')

        print()

        # ====================================================================
        # 2. RELATIONSHIP COUNTS
        # ====================================================================
        print('[2/7] Counting Relationships...')
        print('-' * 70)

        result = session.run('''
            MATCH ()-[r]->()
            RETURN type(r) as rel_type, count(r) as count
            ORDER BY count DESC
        ''')

        rel_data = list(result)
        for record in tqdm(rel_data, desc='Relationship Types'):
            print(f'  • {record["rel_type"]}: {record["count"]:,}')

        total_rels = sum(r['count'] for r in rel_data)
        print(f'\n  Total Relationships: {total_rels:,}')
        print()

        # ====================================================================
        # 3. DATA QUALITY CHECKS
        # ====================================================================
        print('[3/7] Data Quality Checks...')
        print('-' * 70)

        checks = [
            ('Events without Markets', 'MATCH (e:Event) WHERE NOT (e)<-[:PART_OF_EVENT]-() RETURN count(e) as count'),
            ('Markets without Events', 'MATCH (m:Market) WHERE NOT (m)-[:PART_OF_EVENT]->() RETURN count(m) as count'),
            ('Markets without Outcomes', 'MATCH (m:Market) WHERE NOT (m)-[:HAS_OUTCOME]->() RETURN count(m) as count'),
            ('Trades without Users', 'MATCH (t:Trade) WHERE NOT ()-[:PLACED_TRADE]->(t) RETURN count(t) as count'),
            ('Trades without Markets', 'MATCH (t:Trade) WHERE NOT (t)-[:ON_MARKET]->() RETURN count(t) as count'),
            ('Users without Trades', 'MATCH (u:User {role: "trader"}) WHERE NOT (u)-[:PLACED_TRADE]->() RETURN count(u) as count'),
        ]

        all_checks_passed = True
        for check_name, query in tqdm(checks, desc='Quality Checks'):
            result = session.run(query)
            count = result.single()['count']
            status = '✅' if count == 0 else '⚠️'
            print(f'  {status} {check_name}: {count}')
            if count > 0:
                all_checks_passed = False

        if all_checks_passed:
            print('\n  ✅ All quality checks passed!')
        else:
            print('\n  ⚠️  Some quality issues detected (see above)')
        print()

        # ====================================================================
        # 4. MARKET RESOLUTION STATUS
        # ====================================================================
        print('[4/7] Market Resolution Analysis...')
        print('-' * 70)

        result = session.run('''
            MATCH (m:Market)
            RETURN
                sum(CASE WHEN m.resolved = true THEN 1 ELSE 0 END) as resolved_count,
                sum(CASE WHEN m.closed = true THEN 1 ELSE 0 END) as closed_count,
                sum(CASE WHEN m.active = true THEN 1 ELSE 0 END) as active_count,
                count(m) as total_count
        ''')

        record = result.single()
        print(f'  • Resolved Markets: {record["resolved_count"]:,} ({record["resolved_count"]/record["total_count"]*100:.1f}%)')
        print(f'  • Closed Markets: {record["closed_count"]:,} ({record["closed_count"]/record["total_count"]*100:.1f}%)')
        print(f'  • Active Markets: {record["active_count"]:,} ({record["active_count"]/record["total_count"]*100:.1f}%)')
        print(f'  • Total Markets: {record["total_count"]:,}')

        # Show resolved markets with winners.
        print('\n  Sample Resolved Markets:')
        result = session.run('''
            MATCH (m:Market)
            WHERE m.resolved = true AND m.winning_outcome IS NOT NULL
            RETURN m.question as question, m.winning_outcome as winner, m.volume as volume
            ORDER BY m.volume DESC
            LIMIT 3
        ''')

        for record in result:
            print(f'    • "{record["question"][:50]}..."')
            print(f'      Winner: {record["winner"]} | Volume: ${record["volume"]:,.2f}')
        print()

        # ====================================================================
        # 5. TRADING VOLUME ANALYSIS
        # ====================================================================
        print('[5/7] Trading Volume Analysis...')
        print('-' * 70)

        result = session.run('''
            MATCH (t:Trade)
            RETURN
                sum(t.size_usdc) as total_volume,
                avg(t.size_usdc) as avg_trade_size,
                min(t.size_usdc) as min_trade,
                max(t.size_usdc) as max_trade,
                count(t) as trade_count
        ''')

        record = result.single()
        print(f'  • Total Volume: ${record["total_volume"]:,.2f} USDC')
        print(f'  • Average Trade Size: ${record["avg_trade_size"]:,.2f} USDC')
        print(f'  • Smallest Trade: ${record["min_trade"]:,.2f} USDC')
        print(f'  • Largest Trade: ${record["max_trade"]:,.2f} USDC')
        print(f'  • Total Trades: {record["trade_count"]:,}')

        # BUY vs SELL ratio.
        result = session.run('''
            MATCH (t:Trade)
            RETURN
                t.side as side,
                count(t) as count,
                sum(t.size_usdc) as volume
            ORDER BY count DESC
        ''')

        print('\n  Trade Side Distribution:')
        for record in result:
            print(f'    • {record["side"]}: {record["count"]:,} trades (${record["volume"]:,.2f})')
        print()

        # ====================================================================
        # 6. TOP PERFORMERS
        # ====================================================================
        print('[6/7] Top Performers...')
        print('-' * 70)

        # Top traders by volume.
        print('  Top 5 Traders by Volume:')
        result = session.run('''
            MATCH (u:User)-[:PLACED_TRADE]->(t:Trade)
            WITH u, sum(t.size_usdc) as total_volume, count(t) as trade_count
            RETURN u.address as trader, total_volume, trade_count
            ORDER BY total_volume DESC
            LIMIT 5
        ''')

        for idx, record in enumerate(result, 1):
            print(f'    {idx}. {record["trader"][:20]}...')
            print(f'       Volume: ${record["total_volume"]:,.2f} | Trades: {record["trade_count"]}')

        # Top markets by volume.
        print('\n  Top 5 Markets by Volume:')
        result = session.run('''
            MATCH (m:Market)
            RETURN m.question as question, m.volume as volume, m.slug as slug
            ORDER BY m.volume DESC
            LIMIT 5
        ''')

        for idx, record in enumerate(result, 1):
            print(f'    {idx}. {record["question"][:50]}...')
            print(f'       Volume: ${record["volume"]:,.2f}')

        # Top events by volume.
        print('\n  Top 5 Events by Volume:')
        result = session.run('''
            MATCH (e:Event)
            RETURN e.title as title, e.volume as volume, e.category as category
            ORDER BY e.volume DESC
            LIMIT 5
        ''')

        for idx, record in enumerate(result, 1):
            print(f'    {idx}. {record["title"][:40]}...')
            print(f'       Category: {record["category"]} | Volume: ${record["volume"]:,.2f}')
        print()

        # ====================================================================
        # 7. SAMPLE GRAPH PATHS
        # ====================================================================
        print('[7/7] Sample Graph Paths...')
        print('-' * 70)

        # User → Trade → Market → Event path.
        print('  Path: User → Trade → Market → Event')
        result = session.run('''
            MATCH path = (u:User)-[:PLACED_TRADE]->(t:Trade)
                         -[:ON_MARKET]->(m:Market)-[:PART_OF_EVENT]->(e:Event)
            RETURN u.address as user, t.side as side, t.size_usdc as size,
                   t.price as price, m.question as market, e.title as event
            LIMIT 3
        ''')

        for record in result:
            print(f'    • User: {record["user"][:20]}...')
            print(f'      Trade: {record["side"]} ${record["size"]:.2f} @ {record["price"]:.3f}')
            print(f'      Market: "{record["market"][:40]}..."')
            print(f'      Event: "{record["event"][:40]}..."')
            print()

        # Group markets.
        print('  Group Markets (SAME_GROUP):')
        result = session.run('''
            MATCH (m1:Market)-[r:SAME_GROUP]->(m2:Market)
            WITH r.group_id as group_id, collect(DISTINCT m1.group_item_title)[0..3] as sample_options
            RETURN group_id, sample_options, size(sample_options) as option_count
            LIMIT 2
        ''')

        for record in result:
            print(f'    • Group: {record["group_id"][:25]}...')
            print(f'      Options: {", ".join(record["sample_options"])}')
            print()

# Run verification.
verify_database(neo4j_driver)

print('=' * 70)
print('✅ Database Verification Complete!')
print('=' * 70)
